# Import related pacakages

In [1]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [2]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './data/iML1515.xml' 

# Reaction-kcat file.#s-1
# eg. AADDGT,49389.2889
#reaction_kcat_file = "./data/reaction_autopacmen_ori_noadj.csv"
#reaction_kcat_file = "./data/reaction_kappori.csv"
reaction_kcat_file = "./data/reaction_autopacmen_ori_adj.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions and splited isoenzyme. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
#split isoenzyme
model = isoenzyme_split_only(model)
model

Name,iML1515
Memory address,0x029efe5d0908
Number of metabolites,1877
Number of reactions,5918
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [38]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [39]:
reaction_gene_subunit_MW = pd.read_csv("./data/reaction_gene_subunit_MW_old.csv", index_col=0)
reaction_gene_subunit_MW_split_only(reaction_gene_subunit_MW)

In [40]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [41]:
save_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [42]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [43]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
json_output_file="./model/iML1515_irr_enz_constraint_kapp.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1 #kapp data sigma is 1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [44]:
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
fluxes_outfile = './analysis/ECMpy_kappori_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

norm_model = cobra.io.json.load_json_model(json_model_path)
norm_model_pfba_solution = cobra.flux_analysis.pfba(norm_model)
norm_model_pfba_solution_df = norm_model_pfba_solution.to_frame()
norm_model_pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
print(norm_model_pfba_solution_df.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.6472361305642653
0.8769972144269707


# Step4: Calibration parameters

Compare with C13 data

In [12]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ICL', 'PTAr', 'TPI', 'PGI', 'PYK', 'AKGDH', 'ACKr_reverse', 'SUCOAS_reverse', 'FBA', 'PFK', 'PDH', 'ME2']


Select calibration reaction accrording biomass diff

In [13]:
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
enzyme_amount = upperbound
percentage = 0.1#10%
reaction_biomass_outfile = './analysis/reaction_biomass.csv'
select_percentage = 0.8
df_reaction_select = select_calibration_reaction_by_biomass(reaction_kcat_mw_file, json_model_path, enzyme_amount, percentage, reaction_biomass_outfile, select_percentage)
df_reaction_select

,biomass_enz,biomass_orimodel,biomass_diff,biomass_diff_ratio
KARA1_reverse,0.619167,0.876997,0.25783,0.293992


In [14]:
need_change_reaction=list(df_reaction_select.index)
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change.csv"
kcat_data_colect_file="./analysis/reaction_max_df.csv"
change_reaction_list_round1=change_reaction_kcat_by_database_g(need_change_reaction,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['KARA1_reverse']


In [15]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round1.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 0.5 
#sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [16]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round1.json"
fluxes_outfile= './analysis/ECMpy_kapp_adj_round1_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.38308589436103463


In [17]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ICL', 'PTAr', 'TPI', 'PGI', 'AKGDH', 'ACKr_reverse', 'SUCOAS_reverse', 'FBA', 'PFK', 'PDH', 'ME2', 'SUCDi']


Calibration kcat according c13 reaction list

In [18]:
c13reaction_file = './data/C13reaction.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kcat_mw.csv"
f = 0.406
ptot = 0.56 
sigma = 0.5 
enzyme_amount = round(ptot * f * sigma, 3)

c13reaction_selecet=select_calibration_reaction_by_c13(reaction_kcat_mw_file, c13reaction_file, enzyme_amount, 0.1, sigma)
print(c13reaction_selecet)

['GAPD', 'ENO', 'PDH', 'CS', 'AKGDH', 'SUCDi', 'PPC', 'PSERT']


In [19]:
reaction_kcat_mw_file = "./analysis/reaction_kapp_change.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13.csv"
#c13reaction_selecet=['CS','ACONTa','ACONTb','ICDHyr','MALS', 'MDH', 'ICL', 'SUCOAS_reverse', 'SUCDi', 'AKGDH']
kcat_data_colect_file="./analysis/reaction_max_df.csv"
change_reaction_list_round1=change_reaction_kcat_by_database_g(c13reaction_selecet,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['ENO', 'CS', 'AKGDH', 'SUCDi', 'PPC', 'PSERT']


In [20]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round2.json"
f = 0.406
ptot = 0.56 
sigma = 0.5
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [21]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round2.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.4164855885138707


In [22]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ICL', 'TKT2', 'PTAr', 'TPI', 'PGI', 'ACKr_reverse', 'PFK', 'FBA', 'PDH', 'ME2']


Calibration kcat according Enzyme usage 

In [23]:
f = 0.406
ptot = 0.56 
sigma = 0.5
enz_total = round(ptot * f * sigma, 3)
reaction_flux_file = './analysis/ECMpy_kapp_adj_round2_solution_df_pfba.csv' 
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_enz_usage_file = './analysis/ECMpy_kapp_adj_round2_reaction_enz_usage_df.csv'

reaction_enz_usage_df = get_enzyme_usage(enz_total,reaction_flux_file,reaction_kcat_mw_file,reaction_enz_usage_file)
reaction_enz_usage_df.head(22)

,kcat_mw,flux,enz useage,enz ratio
ATPS4rpp_num2,785.767390,40.701289,0.051798,0.454370
PSERT,112.660337,0.692691,0.006148,0.053934
GAPD,2950.918009,9.360443,0.003172,0.027825
PAPSR2_num2,33.787531,0.103073,0.003051,0.026760
KARA2,42.209426,0.121001,0.002867,0.025146
EDA,3885.326983,9.396614,0.002418,0.021215
CYTBO3_4pp,20262.035207,44.303805,0.002187,0.019180
PGL,5091.247502,10.119378,0.001988,0.017435
ACGS,64.093962,0.123193,0.001922,0.016860
SUCOAS_reverse,1808.724502,3.341661,0.001848,0.016206


In [24]:
select_reaction = list(reaction_enz_usage_df[reaction_enz_usage_df['enz ratio']>0.02].index)#more than 1%
print(select_reaction)
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"
kcat_data_colect_file="./analysis/reaction_max_df.csv"

change_reaction_list_round1=change_reaction_kcat_by_database_g(select_reaction,kcat_data_colect_file,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

['ATPS4rpp_num2', 'PSERT', 'GAPD', 'PAPSR2_num2', 'KARA2', 'EDA']
['ATPS4rpp_num2', 'PAPSR2_num2', 'KARA2']


In [25]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_round3.json"
f = 0.406
ptot = 0.56 
sigma = 0.5
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

In [26]:
json_model_path = "./model/iML1515_irr_enz_constraint_adj_round3.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_round3_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

0.6308623145854081


In [27]:
import re
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = []
for eachreaction in enz_model_pfba_solution_select.index:
    if re.search('_num',eachreaction):
        enz_model_pfba_solution_select_id.append(eachreaction.split('_num')[0])
    else:
        enz_model_pfba_solution_select_id.append(eachreaction)

c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['ICL', 'TKT2', 'PTAr', 'TPI', 'PGI', 'PYK', 'ACKr_reverse', 'PFK', 'FBA', 'PDH', 'ME2']


In [54]:
reaction_kcat_mw_file = "./analysis/reaction_kapp_change_c13_enzuse.csv"
reaction_kapp_change_file = "./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
select_reaction=['PDH']
#c13reaction_selecet=['CS','ACONTa','ACONTb','ICDHyr','MALS', 'MDH', 'ICL', 'SUCOAS_reverse', 'SUCDi', 'AKGDH']
change_fold=[20]
change_reaction_list_round1=change_reaction_kcat_by_foldlist(select_reaction,change_fold,reaction_kcat_mw_file,reaction_kapp_change_file)
print(change_reaction_list_round1)

model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
json_output_file="./model/iML1515_irr_enz_constraint_adj_manual.json"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
sigma = 0.5#0.5 
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model_split_isoenzyme_only(model_file, reaction_kcat_mw_file, f, ptot, sigma, lowerbound, upperbound, json_output_file)

json_model_path = "./model/iML1515_irr_enz_constraint_adj_manual.json"
fluxes_outfile = './analysis/ECMpy_kapp_adj_manual_solution_df_pfba.csv'
reaction_kcat_mw_file="./analysis/reaction_kapp_change_c13_enzuse_manual.csv"
enz_model=get_enzyme_constraint_model(json_model_path)
rglcex=enz_model.reactions.get_by_id("GLCDpp_no1")
rglcex.bounds=(0, 0)

enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,fluxes_outfile,reaction_kcat_mw_file)
print(enz_model_pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])

c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

enz_model_pfba_solution_select = enz_model_pfba_solution[enz_model_pfba_solution['fluxes']>0]
enz_model_pfba_solution_select_id = list(enz_model_pfba_solution_select.index)
c13reaction_2_enz_model_diff=list(set(c13reaction).difference(set(enz_model_pfba_solution_select_id)))
print (c13reaction_2_enz_model_diff)

['PDH']
0.8234989839795128
['TKT2', 'ICL', 'ACKr_reverse', 'PTAr', 'ME2']


# Step4: Solveing enzyme concentration constraint by COBRApy.

In [21]:
#run enzyme constraint metabolic model
json_model_path="./model/iML1515_irr_enz_constraint_adj_round2.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.7901116510198356

In [22]:
#run genome-scale metabolic model
norm_model=cobra.io.json.load_json_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']


0.8769972144269698